<a href="https://colab.research.google.com/github/adeepH/CASE-2021-Task-1/blob/main/Notebooks/distilbert_cased_multi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json
sen=[]
l=[]
with open('/content/drive/MyDrive/en-train.json') as f:
  c=0
  for i in f.readlines():
    x=json.loads(i)
    l.append(x['label'])
    sen.append(x['sentence'].strip())

In [ ]:
ssen=[]
sl=[]
with open('/content/drive/MyDrive/es-train.json') as f:
  c=0
  for i in f.readlines():
    x=json.loads(i)
    sl.append(x['label'])
    ssen.append(x['sentence'].strip())
psen=[]
pl=[]
with open('/content/drive/MyDrive/pr-train.json') as f:
  c=0
  for i in f.readlines():
    x=json.loads(i)
    pl.append(x['label'])
    psen.append(x['sentence'].strip())

In [ ]:
x=sen
x.extend(ssen)
x.extend(psen)
y=l
y.extend(sl)
y.extend(pl)

In [ ]:
!pip install transformers

     |████████████████████████████████| 2.3MB 7.2MB/s 
     |████████████████████████████████| 3.3MB 52.0MB/s 
     |████████████████████████████████| 901kB 51.6MB/s 


In [ ]:
from transformers import DistilBertTokenizer
from transformers import TFDistilBertModel

In [ ]:
from sklearn.model_selection import train_test_split
x,x1,y,y1=train_test_split(x,y,test_size=0.2)

In [ ]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-multilingual-cased')

In [ ]:
d=tokenizer(x,padding=True,truncation=True,max_length=128)
d1=tokenizer(x1,padding=True,truncation=True,max_length=128)

In [ ]:
import numpy as np
q1=np.array(d['input_ids'])
q2=np.array(d['attention_mask'])
q3=np.array(d1['input_ids'])
q4=np.array(d1['attention_mask'])

In [ ]:
y=np.array(y)
y1=np.array(y1)

In [ ]:
model = TFDistilBertModel.from_pretrained('distilbert-base-multilingual-cased')

Some layers from the model checkpoint at distilbert-base-multilingual-cased were not used when initializing TFDistilBertModel: ['activation_13', 'vocab_layer_norm', 'vocab_projector', 'vocab_transform']
- This IS expected if you are initializing TFDistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFDistilBertModel were initialized from the model checkpoint at distilbert-base-multilingual-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


In [ ]:
import tensorflow
from tensorflow.compat.v1.keras.layers import CuDNNLSTM

In [ ]:
from tensorflow.keras.layers import Input,Dense,GlobalAveragePooling1D,Dropout,LSTM
from tensorflow.keras.models import Model
inp1=Input(shape=(128,),dtype='int32')
inp2=Input(shape=(128,),dtype='int32')
emb=model(inp1,attention_mask=inp2)[0]
l=CuDNNLSTM(128,return_sequences=True)(emb)
l=GlobalAveragePooling1D()(l)
l=Dense(50,activation='relu')(l)
l=Dropout(0.2)(l)
l=Dense(1,activation='sigmoid')(l)
m=Model(inputs=[inp1,inp2],outputs=l)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experi

In [ ]:
for i in m.layers[:3]:
  i.trainable=False

In [ ]:
import keras
m.compile(optimizer=keras.optimizers.Adam(learning_rate=2e-5),loss='binary_crossentropy',metrics=['accuracy'])
m.fit([q1,q2],y,epochs=5,batch_size=32,validation_data=([q3,q4],y1))

Epoch 1/5
669/669 [==============================] - 136s 195ms/step - loss: 0.2379 - accuracy: 0.8980 - val_loss: 0.2869 - val_accuracy: 0.8761
Epoch 2/5
669/669 [==============================] - 128s 191ms/step - loss: 0.2357 - accuracy: 0.8975 - val_loss: 0.2920 - val_accuracy: 0.8736
Epoch 3/5
669/669 [==============================] - 128s 191ms/step - loss: 0.2351 - accuracy: 0.8979 - val_loss: 0.2999 - val_accuracy: 0.8684
Epoch 4/5
669/669 [==============================] - 128s 191ms/step - loss: 0.2281 - accuracy: 0.9009 - val_loss: 0.2897 - val_accuracy: 0.8748
Epoch 5/5
669/669 [==============================] - 128s 192ms/step - loss: 0.2194 - accuracy: 0.9061 - val_loss: 0.2946 - val_accuracy: 0.8772


In [ ]:
m.save_weights('/content/drive/MyDrive/ditilbert multi model')

In [ ]:
m1=Model(inputs=[inp1,inp2],outputs=l)

In [ ]:
import keras
m1.compile(optimizer=keras.optimizers.Adam(learning_rate=2e-5),loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
m1.load_weights('/content/drive/MyDrive/ditilbert multi model')

In [ ]:
pred=m1.predict([q3,q4])

In [ ]:
p=[0 if i[0]<=0.5 else 1 for i in pred]

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y1,p,digits=3))

              precision    recall  f1-score   support

           0      0.923     0.952     0.937      4357
           1      0.756     0.652     0.700       993

    accuracy                          0.896      5350
   macro avg      0.839     0.802     0.819      5350
weighted avg      0.892     0.896     0.893      5350

